In [20]:
import logging
import pandas as pd
from scrapers.filmladder import FilmladderScraper
from daily_pipeline import assign_ids_cinemas, assign_ids_screenings, extract_unique_movies
from daily_pipeline import add_cineville_tag, add_imdb_links
from save_to_db import save_movies, save_screenings, save_cinemas
# from external.imdb_api import fetch_imdb_metadata
%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
logging.info("Starting daily data pipeline...")

# 1️⃣ Scrape Filmladder (returns two DataFrames)
filmladder = FilmladderScraper()
screenings_df, cinemas_df = filmladder.run()

# 2️⃣ Assign IDs
screenings_df = assign_ids_screenings(screenings_df)
cinemas_df = add_cineville_tag(assign_ids_cinemas(cinemas_df))

# 3️⃣ Extract and fetch IMDb metadata for unique movies
movies_df = extract_unique_movies(screenings_df)

# Add intermediate step: only fetch imdb links / metadata for films that do not yet have one
# movies_df = add_imdb_links(movies_df)

# logging.info("Daily data pipeline completed.")

INFO:root:Starting daily data pipeline...
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.141/chromedriver-mac-x64/chromedriver] found in cache


In [27]:
cinemas_df.to_pickle('temp_data/cinemas.pkl')
movies_df.to_pickle('temp_data/movies.pkl')
screenings_df.to_pickle('temp_data/screenings.pkl')

In [24]:
movies_df = pd.read_pickle('temp_data/movies.pkl')
movies_df = movies_df.drop_duplicates(subset=['imdb_link'], keep='first')

# # 4️⃣ Store data in the database
save_movies(movies_df)
save_screenings(screenings_df)
save_cinemas(cinemas_df)

OperationalError: (sqlite3.OperationalError) no such table: movies
[SQL: SELECT movies.movie_id AS movies_movie_id 
FROM movies]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Example Usage:
# df = pd.read_csv("movies.csv")  # Assuming a CSV with a 'movie_link' column
from scrapers.imdb import IMDBScraper
scraper = IMDBScraper(headless=True)
result_df = scraper.run(movies_df['movie_link'].tolist())

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.127/chromedriver-mac-x64/chromedriver] found in cache
Scraping Progress: 100%|██████████| 160/160 [05:02<00:00,  1.89s/it]


                                            movie_link  \
0    https://www.filmladder.nl/film/favoriten-2024/...   
1    https://www.filmladder.nl/film/merckx-2025/pop...   
2    https://www.filmladder.nl/film/a-complete-unkn...   
3    https://www.filmladder.nl/film/alpha-2024/popu...   
4    https://www.filmladder.nl/film/anora-2024/popu...   
..                                                 ...   
155  https://www.filmladder.nl/film/gallipoli-1981/...   
156  https://www.filmladder.nl/film/marcel-the-shel...   
157  https://www.filmladder.nl/film/memoir-of-a-sna...   
158  https://www.filmladder.nl/film/platoon-1986/po...   
159  https://www.filmladder.nl/film/the-seed-of-the...   

                                 imdb_link  
0    https://www.imdb.com/title/tt31015543  
1    https://www.imdb.com/title/tt35488509  
2    https://www.imdb.com/title/tt11563598  
3    https://www.imdb.com/title/tt28797114  
4    https://www.imdb.com/title/tt28607951  
..                               

In [43]:
df_list = [screenings_df, cinemas_df, movies_df]
for df in df_list:
    print(df.columns)

Index(['cinema_name', 'title', 'year', 'show_datetime', 'ticket_url', 'rating',
       'movie_link', 'poster_url', 'movie_id', 'cinema_id'],
      dtype='object')
Index(['name', 'location', 'address', 'website', 'cinema_id',
       'partnered_with_cineville'],
      dtype='object')
Index(['movie_id', 'title', 'year', 'movie_link', 'imdb_link'], dtype='object')


In [8]:
from scrapers.letterboxd import LetterboxdScraper
lb_scraper = LetterboxdScraper()
watchlist_df = lb_scraper.run()


INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.126/chromedriver-mac-x64/chromedriver] found in cache


In [12]:
cinemas_df = pd.read_pickle('temp_data/cinemas.pkl')
movies_df = pd.read_pickle('temp_data/movies.pkl')
screenings_df = pd.read_pickle('temp_data/screenings.pkl')
watchlist_df = pd.read_pickle('temp_data/watchlist.pkl')

In [14]:
from data_models.base import engine
print(engine.url)

sqlite:///db.sqlite3
